# XY16 dynamical decoupling sequence

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [2]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [3]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [4]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV1'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [29]:
# for single frequency XY16-N:
xy16 = OrderedDict()
xy16['mw_amp'] = 0.25
xy16['mw_freq'] = 100e6
xy16['order'] = 25
xy16['tau_start'] = 251.0e-9
xy16['tau_incr'] = 1.0e-9
xy16['points'] = 22
xy16['measurement_time'] = 40*60
xy16['refocus_interval'] = 3*60
xy16['alternating']=True
xy16['rabi_period']=165.3*1e-9

In [30]:
# generate new XY16 sequence
sequencegeneratorlogic.delete_sequence('XY16_sequence')    
sequencegeneratorlogic.generate_XY16_sequence('XY16_sequence', xy16['rabi_period'], 100e+6, 0.25,
                          xy16['tau_start'], xy16['tau_incr'], xy16['points'], xy16['order'],
                          'a_ch1', 3.0e-6, 2.0, 0.7e-6, 1.0e-6, '', 'd_ch2', xy16['alternating'])
# sample, upload and load waveform
if 'XY16_sequence' not in sequencegeneratorlogic.saved_pulse_sequences:
    sequencegeneratorlogic.log.error('Sequence generation of XY16 in notebook timed out.')
pulsedmasterlogic.sample_sequence('XY16_sequence', True)

while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [23]:
setup['poi'] = poimanagerlogic.active_poi.get_key()

In [24]:
refocus_poi(setup['poi'])

181314.666667 4.788378329750268e-09


True

In [31]:
do_xy16_refocus(setup['poi'])

181314.666667 1.2165125775289343e-09
181314.666667 2.213738233469622e-09
181314.666667 3.948626026492727e-09
181314.666667 3.4443195630416267e-09
181314.666667 4.86862396508218e-10
181314.666667 1.4501334279025781e-09
181314.666667 4.193052301468866e-09
181314.666667 3.0307029277420582e-09
181314.666667 3.5158340211407743e-09
181314.666667 2.703531705654878e-09
181314.666667 4.6813641807838145e-09
181314.666667 1.330162960199333e-09
181314.666667 6.28602036785111e-09


False

In [32]:
# save measurement
pulsedmasterlogic.save_measurement_data('s','XY16-'+str(xy16['order']),True)

In [ ]:
mw_freq = 1393.3e+6 # in MHz
mw_power = -15

pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(mw_freq-0.1e+9)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()

In [9]:
def refocus_poi(poi=None, max_drift=0.5,min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.module_state() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

In [10]:
def do_xy16(save_tag='',load_tag=''):
    
    pulsedmasterlogic.do_fit('No Fit')
    tau_arr = xy16['tau_start'] + np.arange(xy16['points']) * xy16['tau_incr']
    # set parameters in analysis tab
    if xy16['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, 2*xy16['points'], 100e-6, [],xy16['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, xy16['points'], 100e-6, [],xy16['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.21e-6)
    pulsedmasterlogic.analysis_interval_changed(3)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < xy16['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','XY16-'+str(xy16['order'])+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

In [11]:
def do_xy16_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    xy16_total_time = xy16['measurement_time']
    xy16_runtime = 0.0
    xy16['measurement_time'] = xy16['refocus_interval']
    end_measure = do_xy16('xy16_refocus','')
    xy16_runtime += xy16['refocus_interval']
    while xy16_total_time > xy16_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_xy16('xy16_refocus', 'xy16_refocus')
        xy16_runtime += xy16['refocus_interval']
        if end_measure:
            break
    xy16['measurement_time'] = xy16_total_time
    write_to_logfile('xy16_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=xy16_runtime)
    time.sleep(2)
    return end_measure